In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyodbc

In [ ]:
!pip install eurostat

In [ ]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to

In [ ]:
import gensim
import pandas as pd
import numpy as np
import re
import pyodbc

c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")
cursor = c.cursor()


def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df

In [ ]:
#OECD

query =      """SELECT id,article_id,term,url,definition,context,theme,related,related_url,last_update,source_publ FROM ESTAT.V1.OECD_Glossary"""

OECDGlossary = load_table(cursor,query)

query =      """SELECT id,ESTAT_theme,ESTAT_sub_theme,OECD_themes FROM ESTAT.V1.Eurostat_OECD_themes"""

OECDThemes = load_table(cursor,query)

In [ ]:
#Map OECD
listHelper = []
for name, row in OECDGlossary.iterrows():
  helper = ""
  nameTerm = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower()
  nameTerm = nameTerm.replace("____", "").replace("___", "").replace("__", "")
  listHelper.append((str(row[1]), nameTerm))

print(len(listHelper))
f = open("OECD.ttl", "a")
for name, row in OECDGlossary.iterrows():
  helper = ""
  nameTerm = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower()
  nameTerm = nameTerm.replace("____", "").replace("___", "").replace("__", "")
  if nameTerm != "":
    f.write("oecd:" + nameTerm + " rdf:type estat:OECDTerm.\n")
    f.write("oecd:" + nameTerm + " estat:term \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
    f.write("oecd:" + nameTerm + " estat:id \"" + str(row[1]) +"\"^^xsd:integer.\n")
    if str(row[3]) != "":
      f.write("oecd:" + nameTerm + " estat:hasURL \"" + str(row[3]) + "\"^^xsd:anyURI .\n")
    if str(row[4]) != "":
      f.write("oecd:" + nameTerm + " estat:definition \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[4]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
    if str(re.sub("[^a-zA-Z\d%]", ' ', row[5]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) != "":
      f.write("oecd:" + nameTerm + " estat:context \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[5]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")

    if str(row[6]) != "":
      f.write("oecd:" + nameTerm + " estat:hasOECDTheme oecd:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "") +".\n")
      f.write("oecd:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "") + " rdf:type estat:OECDTheme.\n")
      f.write("oecd:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "") + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() + "\"^^xsd:string.\n")

    if str(row[8]) != "":
      helperSplit = str(row[8]).split(",")
      for uri in helperSplit:
        if "?ID=" in str(uri):
          uri = uri.split("?ID=")[1]
          for l in listHelper:
            if str(uri) == str(l[0]):
              f.write("oecd:" + nameTerm + " estat:relatedTerm oecd:" + l[1] + " .\n")

    if str(row[9]) != "":
      f.write("oecd:" + nameTerm + " estat:dateUpdated \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[9]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
    if str(row[10]) != "":
      f.write("oecd:" + nameTerm + " estat:sourcePublication \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[10]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() +"\"^^xsd:string.\n")

  
f.close()


6936


In [ ]:
f = open("OECD.ttl", "a")
for name, row in OECDThemes.iterrows():
  eurostatTheme = str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
  flag = 0
  for theme in row[3].split(";"):
    oecdTheme = str(re.sub("[^a-zA-Z\d%]", ' ', theme).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
    f.write("estatdata:" + eurostatTheme + " rdf:type estat:EurostatTheme.\n")
    f.write("estatdata:" + eurostatTheme + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() + "\"^^xsd:string.\n")
    if oecdTheme != "":
      f.write("estatdata:" + eurostatTheme + " estat:relatedTheme oecd:" + oecdTheme + ".\n")
      flag = 1
  if flag == 1:
    if str(row[2]) not in row[3].split(";"):
      subtheme = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "_").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
      f.write("oecd:" + subtheme + " rdf:type estat:EurostatTheme.\n")
      f.write("oecd:" + subtheme + " estat:term \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
      f.write("estatdata:" + subtheme + " estat:hasSubTheme oecd:" + oecdTheme + ".\n")

f.close()